# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
T. Henning  ->  T. Henning  |  ['T. Henning']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
J. Li  ->  J. Li  |  ['J. Li']
F. Xu  ->  F. Xu  |  ['F. Xu']
Arxiv has 66 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.10751


extracting tarball to tmp_2511.10751...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']


...

/tmp/ipykernel_2865/2822249172.py:52: LatexWarning: 2511.10927 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2511.11179


extracting tarball to tmp_2511.11179...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.11396
extracting tarball to tmp_2511.11396...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.10751-b31b1b.svg)](https://arxiv.org/abs/2511.10751) | **Hydrogen-line profiles from accreting gas giants and their CPDs**  |
|| G.-D. Marleau, et al. -- incl., <mark>T. Henning</mark>, <mark>R. v. Boekel</mark>, <mark>M. Benisty</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| *16 pages, 10 figures, appendixes. Submitted to A&A*|
|**Abstract**|            Few gas giants have been observed in their accretion phase. ELT instruments will have a higher sensitivity and a smaller inner working angle than instruments up to now, allowing detailed characterisation. We study the observability of accreting gas giants with METIS, the first-generation ELT spectrograph with a resolution R = 1e5. We focus on the accretion-tracing hydrogen recombination lines accessible to METIS, mainly Brackett alpha and Pfund-series lines. Our approach is general but we take PDS 70 b as a fiducial case. To calculate high-resolution line profiles, we combine a semianalytical multidimensional description of the flow onto an accreting planet and its circumplanetary disc (CPD) with local NLTE shock-emission calculations. We assume the limiting scenario of no extinction, appropriate for gas giants in gaps, and negligible contribution from magnetospheric accretion. We use resolved photospheric models, and include detector sensitivities to compute required observing times. Both the planet surface and the CPD surface shocks contribute to the total line profile, which is non-Gaussian and much narrower than the free-fall velocity. When the accretion onto PDS 70 b is at its maximum observed strength, the Br a line peak is equal to the photospheric continuum, which is modulated mostly by water features. However, the rotation of the planet broadens the features, making the shock excess stand out. At Br a, already only the continuum of PDS 70 b should yield SNR = 12 in 4 h, and for the fiducial accretion rate, the line peak will have a per-bin SNR = 28 in 4 h. The peak excess should require only about 10 min to reach SNR = 3. Br a is a potent planet formation tracer accessible to ELT/METIS in little integration time. Resolved line profiles will place independent constraints especially on the mass and radius of accreting planets, and help identify the accretion mechanism(s) at work.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.10882-b31b1b.svg)](https://arxiv.org/abs/2511.10882) | **Unveiling the Chemical Complexity and C/O Ratio of the HD 163296 Protoplanetary Disk: Constraints from Multi-line ALMA Observations of Organics, Nitriles, Sulfur-bearing, and Deuterated Molecules**  |
|| P. Kashyap, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Supplement Series (ApJS). 43 pages, 21 figures, 4 tables*|
|**Abstract**|            The physical and chemical conditions within a protoplanetary disk play a crucial role in determining its chemical composition, which is subsequently inherited by any forming planets. To probe these conditions, high-resolution molecular line observations, coupled with modelling, are essential. In this study, we investigate the chemistry of the nearby, massive, and relatively line-rich protoplanetary disk around HD 163296 using high-resolution observations from ALMA across Bands 3, 4, 6, and 7. We constrain the disk-averaged and radial distributions of column density and excitation temperature for the detected molecules using the new retrieval code DRive. The disk chemistry is modelled using the astrochemical code PEGASIS, with variations in the initial elemental C/O ratio. Our modelling, informed by molecular observations of HCO+, DCO+, HCN, DCN, CS, HC3N, H2CO, CH3OH, HNCO, and NH2CHO, allows us to place strong constraints on the C/O ratio, with a best-fit value of 1.1 that is broadly consistent with previous estimates. We present the highest-resolution DCO+ emission map of this disk to date, revealing triple-ringed chemical substructures that closely align with the dust continuum rings. Additionally, our results provide the first and most stringent upper limits on the column densities of NH2CHO and HNCO in this protoplanetary disk, measured at < 7e11 cm-2 and < 1e11 cm-2, respectively. Our chemical models suggest that NH2CHO and HNCO predominantly form on grain surfaces within the disk. However, physico-chemical desorption mechanisms are inefficient at releasing these species into detectable gas-phase abundances, yet they remain promising targets for future ALMA observations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.10805-b31b1b.svg)](https://arxiv.org/abs/2511.10805) | **Mock Observations for the CSST Mission: Main Surveys -- the Mock Catalogue**  |
|| C.-L. Wei, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| *17 pages, 17 figures, accepted in RAA. Our mock catalog is available upon request*|
|**Abstract**|            The Chinese Space Station Survey Telescope (CSST) is a flagship space mission, designed to carry out a large-area sky survey to explore the nature of dark matter and dark energy in the Universe. The onboard multi-band imaging and slitless spectroscopic modules will enable us to obtain photometric data for billions of galaxies and stars, as well as hundreds of millions of spectroscopic measurements, advancing various scientific analyses such as galaxy clustering and weak gravitational lensing. To support the image simulations for the main survey of the CSST mission, we present a mock catalogue of stars and galaxies. For stars, the mock catalogue is generated using either Galaxia or TRILEGAL, both of which provide a range of stellar properties to meet the requirements of CSST image simulations. For galaxies, we built a mock light-cone up to redshift z~3.5 from the cosmological Nbody simulation and populated the mock galaxy catalogue from the dark mater haloes using a semi-analytical galaxy formation model. We then performed a full-sky ray-tracing simulation of weak gravitational lensing to obtain lensing shear at the position of each galaxy in the light-cone. To support both multi-band imaging and slitless spectroscopic simulations, we computed the spectral energy distribution (SED) for each galaxy based on its star formation history using a supervised deep-learning model and determined the magnitudes in each band using the CSST throughputs. Finally, the properties of our mock galaxies include positions, redshifts, stellar masses, shapes, sizes, SEDs, lensing shears and magnifications. We have validated our mock catalogue against observational data and theoretical models, with results showing good overall agreement. The catalogue provides a flexible dataset for the development of CSST image processing and can support a wide range of cosmological analyses within the CSST mission.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.11179-b31b1b.svg)](https://arxiv.org/abs/2511.11179) | **Constraints on the canonical single-field slow-roll inflation model from observations**  |
|| <mark>J. Li</mark>, G.-H. Guo |
|*Appeared on*| *2025-11-17*|
|*Comments*| **|
|**Abstract**|            In this paper we use two methods to constrain the the canonical single-field slow-roll inflation model. The first method exploits the analytic slow-roll-parameter dependence of primordial perturbations, and the second consists of a phenomenological parameterization of the primordial spectra of both scalar and tensor perturbations. We constrain the slow-roll parameters directly by adopting the latest datasets, including Planck satellite data, BICEP2/Keck data and Baryon Acoustic Oscillation data. An advantage of this method is that we can work out the predictions of single-field slow-roll inflation model by using these constrained slow-roll parameters. We illustrate the predictions of the parameters characterizing the scalar power spectrum and constrain some inflation models. We find that the inflation model with monomial potential is disfavored, and the inflation models with a concave potential, such as the Starobinsky inflation model, brane inflation model are preferred. From the constraints on the slow-roll parameters, the derived tensor spectral index in the single-field slow-roll inflation model is quite small, namely $|n_t|\lesssim 4.7\times 10^{-3}$ which will be very difficult to be measured by CMB data only in the future, and the absolute value of derived running of tensor spectral index is not larger than $1.56\times 10^{-4}$ at $95\%$ confidence level.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.11396-b31b1b.svg)](https://arxiv.org/abs/2511.11396) | **Modeling the Multi-Wavelength Afterglow of Short Gamma-Ray Bursts with a Plateau Phase**  |
|| C. Deng, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| *19 pages, 6 figures, 1 table*|
|**Abstract**|            Short gamma-ray bursts (GRBs) exhibiting a plateau phase provide valuable insights into the post-merger activity of their central engines. Although the physical origin of the plateau remains uncertain, the magnetar energy injection model offers a compelling explanation that reproduces the observed temporal and luminosity features. However, previous studies relying solely on X-ray data have suffered from strong parameter degeneracies when constraining the magnetar parameters. Here we perform broadband afterglow modeling on seven short GRBs with plateau features by combining X-ray, optical, and radio observations within the framework of the magnetar energy injection model. Key model parameters are derived by using the Markov Chain Monte Carlo method. It is found that the energy injection substantially modifies the afterglow dynamics in most events. Compared with X-ray--only analyses, our broadband modeling yields systematically a lower magnetic field strength and a shorter spin period for the central magnetar, corresponding to a higher injection luminosity. The study clearly shows that incorporating multi-wavelength data effectively alleviates the degeneracy between the magnetar parameters and X-ray radiative efficiency. In addition, the distribution of our short GRBs differs markedly from long GRBs when they are plotted on the initial Lorentz factor versus gamma-ray energy plane. This offset, consistent with the observed harder spectrum of short GRBs, may serve as a useful diagnostic for investigating the progenitor as larger samples are available.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.10927-b31b1b.svg)](https://arxiv.org/abs/2511.10927) | **PRIMA General Observer Science Book Volume 2**  |
|| A. Moullet, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| *A. Moullet, D. Burgarella, T. Kataria, H. Beuther, C. Battersby, M. Cheng, T. Essinger-Hileman, H. Inami, E. Mills, T. Nagao, S. Unwin are the editors of the GO Science Book. 664 pages*|
|**Abstract**|            This PRobe far-Infrared Mission for Astrophysics (PRIMA) mission concept is a proposed mission to NASA's Astrophysics Probe Explorer (APEX) call. The concept features a cryogenically cooled 1.8 m diameter telescope, and is designed to carry two science instruments covering the 24 to 264 $\mu$m wavelength range: an imaging polarimeter (PRIMAger) and a spectrometer (FIRESS). The majority of PRIMA's time (75%) will be open to observations proposed by the community (General Observer science / GO), and all of data will be publicly available for archival research (Guest Investigator science / GI). Following up on the successful community engagement created by the first volume of the GO PRIMA Science Book (arXiv:2310.20572), Volume 2 gathers 120 new and updated contributed science cases which could be performed within the context of the PRIMA GO/GI program. This volume reflects the strong development of the community interest, awareness and involvement in PRIMA, and further develops how PRIMA's unprecedented capabilities can be leveraged for an impactful and innovative GO/GI program covering most areas of astrophysics and over 90% of the scientific questions and discovery areas in the Astro2020 decadal survey.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.10825-b31b1b.svg)](https://arxiv.org/abs/2511.10825) | **ALMAGAL V. Relations between the core populations and the parent clump physical properties**  |
|| D. Elia, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-11-17*|
|*Comments*| **|
|**Abstract**|            Context. The fragmentation of massive molecular clumps into smaller, potentially star-forming cores plays a key role in the processes of high-mass star formation. The ALMAGAL project offers high-resolution data to investigate these processes across various evolutionary stages in the Galactic plane. Aims. This study aims at correlating the fragmentation properties of massive clumps, obtained from ALMA observations, with their global physical parameters (e.g., mass, surface density, and temperature) and evolutionary indicators (such as luminosity-to-mass ratio and bolometric temperature) obtained from Herschel observations. It seeks to assess whether the cores evolve in number and mass in tandem with their host clumps, and to determine the possible factors influencing the formation of massive cores (M > 24M_\odot). Methods. We analyzed the masses of 6348 fragments, estimated from 1.4 mm continuum data for 1007 ALMAGAL clumps. Leveraging this unprecedentedly large data set, we evaluated statistical relationships between clump parameters, estimated over about 0.1 pc scales, and fragment properties, corresponding to scales of a few 1000 au, while accounting for potential biases related to distance and observational resolution. Our results were further compared with predictions from numerical simulations. Results. The fragmentation level correlates preferentially with clump surface density, supporting a scenario of density-driven fragmentation, whereas it does not show any clear dependence on total clump mass. Both the mass of the most massive core and the core formation efficiency show a broad range and increase on average by an order of magnitude in the intervals spanned by evolutionary indicators such as clump dust temperature and the luminosity-to-mass ratio. This suggests that core growth continues throughout the clump evolution, favoring clump-fed over core-fed theoretical scenarios.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error string index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Bra_kStern_mitRauschen.png', 'tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Bra_kStern_mitRauschen_Zoom70_lin.png', 'tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfb_kStern_mitRauschen_Zoom70_lin.png', 'tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfg_kStern_mitRauschen_Zoom70_lin.png', 'tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfd_kStern_mitRauschen_Zoom70_lin.png', 'tmp_2511.10751/./Flambda_Teff_CIFIST_Bra_logg3.5.normPhot.png', 'tmp_2511.10751/./Winkelschema_5.png']
copying  tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Bra_kStern_mitRauschen.png to _build/html/
copying  tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Tef

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Bildbreite}[1]{#1}$
$\newcommand{\auskommentiert}[1]$
$\newcommand{\MSonne}{{M_{\odot}}}$
$\newcommand{\RSonne}{{R_{\odot}}}$
$\newcommand{\LSonne}{{L_{\odot}}}$
$\newcommand{\FOb}{\ensuremath{\mathcal{F}}\xspace}$
$\newcommand{\FlOb}{\ensuremath{\mathcal{F}_\lambda}\xspace}$
$\newcommand{\LAkk}{\ensuremath{{L_{\textnormal{acc}}}}\xspace}$
$\newcommand{\FAkk}{\ensuremath{{F_{\textnormal{acc}}}}\xspace}$
$\newcommand{\Rphot}{{R_{\textnormal{phot}}}}$
$\newcommand{\RAkk}{{R_{\textnormal{acc}}}}$
$\newcommand{\RHill}{{R_{\textnormal{Hill}}}}$
$\newcommand{\RBondi}{{R_{\textnormal{Bondi}}}}$
$\newcommand{\kLiss}{{k_{\textnormal{Liss}}}}$
$\newcommand{\Tint}{{T_{\textnormal{int}}}}$
$\newcommand{\Teff}{\ensuremath{T_{\textnormal{eff}}}\xspace}$
$\newcommand{\Teffmin}{\ensuremath{T_{\textnormal{eff, min}}}\xspace}$
$\newcommand{\Teffmax}{\ensuremath{T_{\textnormal{eff, max}}}\xspace}$
$\newcommand{\TSchb}{\ensuremath{T_{\textnormal{disc}}}\xspace}$
$\newcommand{\RSchb}{\ensuremath{R_{\textnormal{disc}}}\xspace}$
$\newcommand{\FSchb}{\ensuremath{F^{\textnormal{disc}}}\xspace}$
$\newcommand{\fAoXX}{\ensuremath{f_{\textrm{\citetalias{Aoyama+2020}}}}\xspace}$
$\newcommand{\muzpSch}{\ensuremath{\mu_{\textrm{CPD}}}\xspace}$
$\newcommand{\Rzent}{\ensuremath{{R_{\textrm{cent}}}}\xspace}$
$\newcommand{\fzent}{\ensuremath{{f_{\textrm{cent}}}}\xspace}$
$\newcommand{\vFfinfty}{\ensuremath{{\varv_{\textrm{ff}, \infty}}}\xspace}$
$\newcommand{\thzpSch}{\ensuremath{\theta_{\textrm{CPD}}}\xspace}$
$\newcommand{\hzpSch}{\ensuremath{h_{\textrm{CPD}}}\xspace}$
$\newcommand{\PLUTO}{{\texttt{PLUTO}}}$
$\newcommand{\mkmk}{{\texttt{Makemake}}}$
$\newcommand{\belt}{{\texttt{Belt}}}$
$\newcommand{\btsettl}[1][]{{\texttt{BT-Settl#1}}}$
$\newcommand{\heracles}{{\texttt{heracles}}}$
$\newcommand{\rapax}{{\texttt{Rapax 21}}}$
$\newcommand{\mesa}{{\texttt{mesa}}}$
$\newcommand{\compl}{{\texttt{Completo 21}}}$
$\newcommand{\ja}{\lstinline|YES|}$
$\newcommand{\nein}{\lstinline|NO|}$
$\newcommand{\mum}{\ensuremath{\upmu\mathrm{m}}\xspace}$
$\newcommand{\Vekt}[1]{\mathbf{#1}}$
$\newcommand{\eqsep}{\;\;\;}$
$\newcommand{\K}[1]$
$\newcommand{\MPktEinhJ}{\MJ \mbox{yr}^{-1}}$
$\newcommand{\MPktEinhS}{\MSonne \mbox{yr}^{-1}}$
$\newcommand{\Lbol}{{L_{\textnormal{bol}}}}$
$\newcommand{\sigSB}{{\sigma}}$
$\newcommand{\MStern}{\ensuremath{M_{\star}}\xspace}$
$\newcommand{\MSternen}{{M_{\star, 1}}}$
$\newcommand{\RStern}{\ensuremath{R_{\star}}\xspace}$
$\newcommand{\TZerst}{{T_{\textnormal{dest}}}}$
$\newcommand{\RZerst}{{R_{\textnormal{dest}}}}$
$\newcommand{\vFf}{{v_{\textnormal{ff}}}}$
$\newcommand{\fred}{{f_{\textnormal{red}}}}$
$\newcommand{\fredlmbd}{{f_{\textnormal{red}, \lambda}}}$
$\newcommand{\fFuell}{\ensuremath{f_{\textnormal{fill}}}\xspace}$
$\newcommand{\Erad}{{E_{\textnormal{rad}}}}$
$\newcommand{\Trad}{{T_{\textnormal{rad}}}}$
$\newcommand{\Eradlmbd}{{E_{\textnormal{rad}, \lambda}}}$
$\newcommand{\Tradlmbd}{{T_{\textnormal{rad}, \lambda}}}$
$\newcommand{\TPop}{{T_{\textnormal{pop}}}}$
$\newcommand{\TGas}{{T_{\textnormal{gas}}}}$
$\newcommand{\rmin}{\ensuremath{r_{\mathrm{min}}}\xspace}$
$\newcommand{\rmax}{\ensuremath{r_{\mathrm{max}}}\xspace}$
$\newcommand{\thSch}{\ensuremath{{\theta_{\mathrm{shock}}}}\xspace}$
$\newcommand$
$\newcommand{\lmbnewcommandf}{\ensuremath{\lambda_{\textnormal{eff}}}\xspace}$
$\newcommand{\Weff}{\ensuremath{W_{\textnormal{eff}}}\xspace}$
$\newcommand{\neuThH}[1]{\textcolor{blue!50!green}{#1}\xspace}$
$\newcommand{\neuMB}[1]{\textcolor{green!50!black}{#1}\xspace}$
$\newcommand{\neuRvB}[1]{\textcolor{red!30!yellow!20!black!30!red}{#1}\xspace}$
$\newcommand{\neuYA}[1]{\textcolor{blue!60!black}{#1}\xspace}$
$\newcommand{\neuGDM}[1]{\textcolor{red!50!blue}{#1}\xspace}$
$\newcommand{\neuGDMII}[1]{\textcolor{red!70!blue}{#1}\xspace}$
$\newcommand{\neuThH}[1]{#1}$
$\newcommand{\neuMB}[1]{#1}$
$\newcommand{\neuRvB}[1]{#1}$
$\newcommand{\neuYA}[1]{#1}$
$\newcommand{\neuGDM}[1]{#1}$
$\newcommand{\neuGDMII}[1]{#1}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\entspr}{ \widehat{=} }$
$\newcommand{\mH}{m_\mathrm{H}}$
$\newcommand{\me}{m_\mathrm{e}}$
$\newcommand{\kB}{k_\mathrm{B}}$
$\newcommand{\MJ}{M_{\mathrm{J}}}$
$\newcommand{\RJ}{R_{\mathrm{J}}}$
$\newcommand{\ME}{M_{\mathrm{E}}}$
$\newcommand{\Lkin}{L_\mathrm{kin}}$
$\newcommand{\Lya}{Ly \alpha\xspace}$
$\newcommand{\Ha}{H \alpha\xspace}$
$\newcommand{\Hb}{H \beta\xspace}$
$\newcommand{\Baa}{\Ha\xspace}$
$\newcommand{\nl}{\ensuremath{n_\ell}\xspace}$
$\newcommand{\Paa}{Pa \alpha\xspace}$
$\newcommand{\Pab}{Pa \beta\xspace}$
$\newcommand{\Pag}{Pa \gamma\xspace}$
$\newcommand{\Bra}{Br \alpha\xspace}$
$\newcommand{\BraM}{\mathrm{Br} \alpha}$
$\newcommand{\Brg}{Br \gamma\xspace}$
$\newcommand{\Pfa}{Pf \alpha\xspace}$
$\newcommand{\Pfb}{Pf \beta\xspace}$
$\newcommand{\PfbM}{\mathrm{Pf} \beta}$
$\newcommand{\Pfg}{Pf \gamma\xspace}$
$\newcommand{\PfgM}{\mathrm{Pf} \gamma}$
$\newcommand{\Pfd}{Pf \delta\xspace}$
$\newcommand{\Pfe}{Pf \varepsilon\xspace}$
$\newcommand{\Hua}{Hu \alpha\xspace}$
$\newcommand{\Ks}{K_{\mathrm{s}}\xspace}$
$\newcommand{\LkCa}{LkCa 15\xspace}$
$\newcommand{\LkCab}{LkCa 15 b\xspace}$
$\newcommand{\PDS}{PDS 70\xspace}$
$\newcommand{\PDSA}{PDS 70 A\xspace}$
$\newcommand{\PDSb}{PDS 70 b\xspace}$
$\newcommand{\PDSc}{PDS 70 c\xspace}$
$\newcommand{\PDSbc}{PDS 70 b and~c\xspace}$
$\newcommand{\PDSd}{PDS 70 d\xspace}$
$\newcommand{\Dlrmb}{Delorme 1 (AB)b\xspace}$
$\newcommand{\WISPb}{WISPIT 2 b\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km} \mathrm{s^{-1}}}\xspace}$
$\newcommand{\MdotU}{\ME \mathrm{yr}^{-1}}$
$\newcommand{\MdotUJ}{\MJ \mathrm{yr}^{-1}}$
$\newcommand{\MdotUS}{\MSun \mathrm{yr}^{-1}}$
$\newcommand{\MPkt}{\ensuremath{\dot{M}}\xspace}$
$\newcommand{\MPktzpSch}{\dot{M}_{\mathrm{CPD}}}$
$\newcommand{\dMdtXCVII}{\ensuremath{\dot{M}_{97\%}}\xspace}$
$\newcommand{\lgMd}{{\mathrm{lg}\Mdot_2}}$
$\newcommand{\MP}{\ensuremath{M_{\mathrm{p}}}\xspace}$
$\newcommand{\RP}{\ensuremath{R_{\mathrm{p}}}\xspace}$
$\newcommand{\ffill}{f_{\mathrm{fill}}}$
$\newcommand{\ff}{\ffill}$
$\newcommand{\Lcont}{L_{\mathrm{cont}}}$
$\newcommand{\LHa}{\ensuremath{{L_{\textnormal{H} \alpha}}}\xspace}$
$\newcommand{\LHamax}{{L_{\textnormal{H} \alpha, \textnormal{max}}}}$
$\newcommand{\FHa}{{F_{\textnormal{H} \alpha}}}$
$\newcommand{\FHamod}{{F_{\textnormal{H} \alpha}^{\textnormal{mod}}}}$
$\newcommand{\FHb}{{F_{\textnormal{H} \beta}}}$
$\newcommand{\FBra}{{F_{\textnormal{Br} \alpha}}}$
$\newcommand{\LBra}{\ensuremath{L_{\textnormal{Br} \alpha}}\xspace}$
$\newcommand{\LBrakumul}{\ensuremath{L_{\textnormal{Br} \alpha}^{\textnormal{cml}}}\xspace}$
$\newcommand{\LBratotPl}{\ensuremath{L_{\textnormal{Br} \alpha}^{\textnormal{tot, plan.}}}\xspace}$
$\newcommand{\ABra}{{A_{\textnormal{Br} \alpha}}}$
$\newcommand{\FPab}{{F_{\textnormal{Pa} \beta}}}$
$\newcommand{\fluxratobs}{\varphi_{\textnormal{obs}}}$
$\newcommand{\fluxratth}{\varphi_{\textnormal{theo, surf}}}$
$\newcommand{\fluxratthred}{\varphi_{\textnormal{theo, rednd}}}$
$\newcommand{\pUV}{p_{\textnormal{UV}}}$
$\newcommand{\pNIR}{p_{\textnormal{NIR}}}$
$\newcommand{\LLinie}{\ensuremath{L_{\mathrm{line}}}\xspace}$
$\newcommand{\FLinie}{F_{\mathrm{line}}}$
$\newcommand{\tauHa}{{\langle\tau_{\textnormal{H} \alpha}\rangle}}$
$\newcommand{\cs}{c_{\mathrm{s}}}$
$\newcommand{\Mach}{\mathcal{M}}$
$\newcommand{\Pram}{P_{\textnormal{ram}}}$
$\newcommand{\yt}{y_\mathrm{t}}$
$\newcommand{\nH}{n_\mathrm{H}}$
$\newcommand{\XHH}{{X_{\mathrm{H}_2}}}$
$\newcommand{\XHI}{{X_{\mathrm{H}}}}$
$\newcommand{\XHII}{{X_{\mathrm{H}^+}}}$
$\newcommand{\AR}{A_R}$
$\newcommand{\AU}{A_U}$
$\newcommand{\AHa}{\ensuremath{A_{\mathrm{H} \alpha}}\xspace}$
$\newcommand{\AV}{A_V}$
$\newcommand{\RV}{R_V}$
$\newcommand{\AH}{A_H}$
$\newcommand{\AK}{A_K}$
$\newcommand{\He}{\mathcal{H}_\mathrm{e}}$
$\newcommand{\HeNull}{\mathcal{H}_\mathrm{e, 0}}$
$\newcommand{\vth}{v_{\textnormal{th}}}$
$\newcommand{\nmax}{n_{\mathrm{max}}}$
$\newcommand{\DF}{\Delta F}$
$\newcommand{\lmbdHa}{\lambda_{\mathrm{H} \alpha}}$
$\newcommand{\lmbdHb}{\lambda_{\mathrm{H} \beta}}$
$\newcommand{\lmbdBra}{\lambda_{\mathrm{Br} \alpha}}$
$\newcommand{\taudHa}{\tau_{\mathrm{dust, H} \alpha}}$
$\newcommand{\kappaStbint}{\kappa_{\bullet, \mathrm{H} \alpha}}$
$\newcommand{\kapStbfpg}{\kappa_{\mathrm{dust, H} \alpha}}$
$\newcommand{\kapmax}{\kappa_{\mathrm{max}}}$
$\newcommand{\Sigmad}{\Sigma_{\mathrm{dust}}}$
$\newcommand{\Sigwhered}{\tilde{\Sigma}_{\textrm{gas}}}$
$\newcommand{\vK}{v_{\mathrm{K}}}$
$\newcommand{\taumax}{{\tau_{\mathrm{max}}}}$
$\newcommand{\fC}{f_\mathrm{C}}$
$\newcommand{\fSi}{f_\mathrm{sil}}$
$\newcommand{\fmax}{f_\mathrm{max}}$
$\newcommand{\finf}{f_{\mathrm{inf}}}$
$\newcommand{\thmin}{\ensuremath{\theta_\mathrm{min}}\xspace}$
$\newcommand{\thmax}{\ensuremath{\theta_\mathrm{max}}\xspace}$
$\newcommand{\varthmaxP}{\ensuremath{\vartheta_\mathrm{max, p}}\xspace}$
$\newcommand{\varthmaxSch}{\ensuremath{\vartheta_\mathrm{max, CPD}}\xspace}$
$\newcommand{\varmu}{\ensuremath{\tilde{\mu}}}$
$\newcommand{\varmumin}{\ensuremath{\varmu_\mathrm{min}}\xspace}$
$\newcommand{\thetaPolebene}{\ensuremath{\theta_{\varphi=\phi}}\xspace}$
$\newcommand{\Dlmbdbin}{(\Delta\lambda)_\textrm{bin}}$
$\newcommand{\tBeob}{\ensuremath{t_\textrm{obs}}\xspace}$
$\newcommand{\FlmbdEmpf}{\ensuremath{F_\lambda^{\mathrm{sens}}}\xspace}$
$\newcommand{\relDvBreite}{\ensuremath{\mathfrak{v}}\xspace}$
$\newcommand{\fkrit}{\ensuremath{f_{\mathrm{crit}}}\xspace}$
$\newcommand{\vZerf}{\ensuremath{v_{\mathrm{brkp}}}\xspace}$
$\newcommand{\FWHM}{\ensuremath{\mathrm{FWHM}}\xspace}$
$\newcommand{\SNR}{\ensuremath{\mathrm{SNR}}\xspace}$
$\newcommand{\nuabschn}{\ensuremath{\tilde{\nu}_{\mathrm{cut}}}\xspace}$
$\newcommand{\vSyst}{\ensuremath{v_{\mathrm{syst}}}\xspace}$
$\newcommand{\tauZeitskala}{t}$
$\newcommand{\la}{\lesssim}$
$\newcommand{\ga}{\gtrsim}$
$\newcommand{\upi}{\pi}$
$\newcommand{\umu}{\upmu}$
$\newcommand{\arcsec}{^\prime ^\prime}$
$\newcommand{◦}{{\mbox{\textdegree}}}$
$\newcommand\ref{@jnl#1}$
$\newcommand{\aj}{\ref@jnl{AJ}}$
$\newcommand{\actaa}{\ref@jnl{Acta Astron.}}$
$\newcommand{\araa}{\ref@jnl{ARA\&A}}$
$\newcommand{\apj}{\ref@jnl{ApJ}}$
$\newcommand{\apjl}{\ref@jnl{ApJ}}$
$\newcommand{\apjs}{\ref@jnl{ApJS}}$
$\newcommand{\ao}{\ref@jnl{Appl.~Opt.}}$
$\newcommand{\apss}{\ref@jnl{Ap\&SS}}$
$\newcommand{\aap}{\ref@jnl{A\&A}}$
$\newcommand{\aapr}{\ref@jnl{A\&A~Rev.}}$
$\newcommand{\aaps}{\ref@jnl{A\&AS}}$
$\newcommand{\azh}{\ref@jnl{AZh}}$
$\newcommand{\baas}{\ref@jnl{BAAS}}$
$\newcommand{\bac}{\ref@jnl{Bull. astr. Inst. Czechosl.}}$
$\newcommand{\caa}{\ref@jnl{Chinese Astron. Astrophys.}}$
$\newcommand{\cjaa}{\ref@jnl{Chinese J. Astron. Astrophys.}}$
$\newcommand{\icarus}{\ref@jnl{Icarus}}$
$\newcommand{\jcap}{\ref@jnl{J. Cosmology Astropart. Phys.}}$
$\newcommand{\jrasc}{\ref@jnl{JRASC}}$
$\newcommand{\memras}{\ref@jnl{MmRAS}}$
$\newcommand{\mnras}{\ref@jnl{MNRAS}}$
$\newcommand{\na}{\ref@jnl{New A}}$
$\newcommand{\nar}{\ref@jnl{New A Rev.}}$
$\newcommand{\pra}{\ref@jnl{Phys.~Rev.~A}}$
$\newcommand{\prb}{\ref@jnl{Phys.~Rev.~B}}$
$\newcommand{\prc}{\ref@jnl{Phys.~Rev.~C}}$
$\newcommand{\prd}{\ref@jnl{Phys.~Rev.~D}}$
$\newcommand{\pre}{\ref@jnl{Phys.~Rev.~E}}$
$\newcommand{\prl}{\ref@jnl{Phys.~Rev.~Lett.}}$
$\newcommand{\pasa}{\ref@jnl{PASA}}$
$\newcommand{\pasp}{\ref@jnl{PASP}}$
$\newcommand{\pasj}{\ref@jnl{PASJ}}$
$\newcommand{\qjras}{\ref@jnl{QJRAS}}$
$\newcommand{\rmxaa}{\ref@jnl{Rev. Mexicana Astron. Astrofis.}}$
$\newcommand{\rnaas}{\ref@jnl{RNAAS}}$
$\newcommand{\skytel}{\ref@jnl{S\&T}}$
$\newcommand{\solphys}{\ref@jnl{Sol.~Phys.}}$
$\newcommand{\sovast}{\ref@jnl{Soviet~Ast.}}$
$\newcommand{\ssr}{\ref@jnl{Space~Sci.~Rev.}}$
$\newcommand{\zap}{\ref@jnl{ZAp}}$
$\newcommand{\nat}{\ref@jnl{Nature}}$
$\newcommand{\natas}{\ref@jnl{Nat.~Ast.}}$
$\newcommand{\iaucirc}{\ref@jnl{IAU~Circ.}}$
$\newcommand{\aplett}{\ref@jnl{Astrophys.~Lett.}}$
$\newcommand{\apspr}{\ref@jnl{Astrophys.~Space~Phys.~Res.}}$
$\newcommand{\bain}{\ref@jnl{Bull.~Astron.~Inst.~Netherlands}}$
$\newcommand{\fcp}{\ref@jnl{Fund.~Cosmic~Phys.}}$
$\newcommand{\gca}{\ref@jnl{Geochim.~Cosmochim.~Acta}}$
$\newcommand{\grl}{\ref@jnl{Geophys.~Res.~Lett.}}$
$\newcommand{\jcp}{\ref@jnl{J.~Chem.~Phys.}}$
$\newcommand{\jgr}{\ref@jnl{J.~Geophys.~Res.}}$
$\newcommand{\jqsrt}{\ref@jnl{J.~Quant.~Spec.~Radiat.~Transf.}}$
$\newcommand{\memsai}{\ref@jnl{Mem.~Soc.~Astron.~Italiana}}$
$\newcommand{\nphysa}{\ref@jnl{Nucl.~Phys.~A}}$
$\newcommand{\physrep}{\ref@jnl{Phys.~Rep.}}$
$\newcommand{\physscr}{\ref@jnl{Phys.~Scr}}$
$\newcommand{\planss}{\ref@jnl{Planet.~Space~Sci.}}$
$\newcommand{\procspie}{\ref@jnl{Proc.~SPIE}}$
$\newcommand{\ptp}{\ref@jnl{Prog.~Th.~Phys.}}$
$\newcommand{\natas}{\ref@jnl{NatAs}}$
$\newcommand{\amjm}{\ref@jnl{AmJM}}$
$\newcommand{\fpg}{f_{\textrm{d/g}}}$
$\newcommand{\amax}{a_{\rm max}}$
$\newcommand{\amin}{a_{\rm min}}$
$\newcommand{\Lint}{\ensuremath{L_{\mathrm{int}}}\xspace}$
$\newcommand{\Lphot}{\ensuremath{L_{\mathrm{phot}}}\xspace}$
$\newcommand{\Lshock}{\ensuremath{L_{\mathrm{shock}}}\xspace}$
$\newcommand{\Hi}{\textnormal{H \textsc{i}}\xspace}$
$\newcommand{\Tmax}{T_{\rm max}}$
$\newcommand{\dAproj}{\ensuremath{{\rm d}A_{\mathrm{proj}}}\xspace}$
$\newcommand{\Aproj}{\ensuremath{A_{\mathrm{proj}}}\xspace}$
$\newcommand{\FlmbdPhot}{\ensuremath{F_{\mathrm{\lambda, phot}}}\xspace}$
$\newcommand{\FlmbdSchock}{\ensuremath{F_{\mathrm{\lambda, shock}}}\xspace}$
$\newcommand{\HFh}{0.187}$</div>



<div id="title">

# Hydrogen-line profiles from accreting gas giants and their CPDs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.10751-b31b1b.svg)](https://arxiv.org/abs/2511.10751)<mark>Appeared on: 2025-11-17</mark> -  _16 pages, 10 figures, appendixes. Submitted to A&A_

</div>
<div id="authors">

G.-D. Marleau, et al. -- incl., <mark>T. Henning</mark>, <mark>R. v. Boekel</mark>, <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Many mature gas giants are known but only a few have been $\neuThH{observed in their accretion phase}$ . However, Extremely Large Telescope (ELT) instruments will have a vastly higher sensitivity and a smaller inner working angle than instruments up to now, making searches more productive and allowing for detailed characterisation. We study the observability of accreting gas giants with METIS, the first-generation ELT spectrograph featuring a resolution $R\sim10^5$ . We focus on the accretion-tracing hydrogen $\neuThH{recombination lines}$ accessible to METIS, mainly $\Bra$ and Pfund-series lines. $\neuThH{Our approach is general but we}$ take $\PDSb$ as a fiducial case. To calculate high-resolution line profiles, we combine a semianalytical multidimensional description of the flow onto an accreting planet and its circumplanetary disc (CPD) with local non-equilibrium shock-emissioncalculations.We assume the limiting scenario of no extinction, $\neuThH{appropriate for gas giants in gaps}$ , and negligible contribution from magnetospheric accretion columns.We use resolved atmospheric models to estimate the photospheric signal, and use simulated detector sensitivities to compute required observing times. Both the planet surface and the CPD surface shocks contribute to the total line profile, which is non-Gaussian and much narrower, $\neuThH{in Doppler velocity space,}$ than the free-fall velocity.When the accretion onto $\PDSb$ is at its maximum observed strength, the $\Bra$ line peak is equal to the photospheric continuum $\neuMB{, which is modulated mostly by water features, with a quantitative dependence on the choice of line list.}$ However, the rotation of the planet significantly $\neuMB{broadens}$ the features, helping the shock excess stand out. $\neuRvB{At \Bra, already only the continuum of \PDSb should yield $\SNR=12$ in 4 h, and with the baseline adopted accretion rate, the line peak will have a per-bin $\SNR\approx28$ in 4 h, or $\SNR\approx3$ in 3 min. The peak excess should require only about 10 min to reach $\SNR\approx3$.}$ $\Bra$ is a potent planet formation tracer and is accessible to ELT/METIS in little integration time. Resolved line profiles will place independent constraints especially on the mass and radius of an accreting planet, and help identify the accretion mechanism(s) at work.

</div>

<div id="div_fig1">

<img src="tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Bra_kStern_mitRauschen.png" alt="Fig15.1" width="20%"/><img src="tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Bra_kStern_mitRauschen_Zoom70_lin.png" alt="Fig15.2" width="20%"/><img src="tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfb_kStern_mitRauschen_Zoom70_lin.png" alt="Fig15.3" width="20%"/><img src="tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfg_kStern_mitRauschen_Zoom70_lin.png" alt="Fig15.4" width="20%"/><img src="tmp_2511.10751/./alles_5.0MJ_2.0RJ_Ink50_thzpSch77_Sigtat2.0e-01_Teff14.0-3.5_vsini10_3sig4h_Pfd_kStern_mitRauschen_Zoom70_lin.png" alt="Fig15.5" width="20%"/>

**Figure 15. -** 
Predicted spectrum of $\PDS$b at $\Bra$
and noise for a 4-h integration (_black line_ with 1-$\$sigma_grey_ errorbars) on the detector wavelength grid.
We sum for this the photosphere (CIFIST with $\Teff=1400$ K, $\log g=3.5$ broadened to $v\sin i=10$ $\kms$; _pale and dark orange_, respectively)
and the shock emission (as in Figure \ref{Abb:Prof}; _dark red_).
The _grey line_ shows the 4-h, 3-$\sigma$ METIS sensitivity curve.
_Top panel_: a portion, centred on $\Bra$, of the range available at once.
Small ticks indicate resonances of $H_2$O (_pale blue_; \citealp{barber06}: "BT2") and $H_2$S (_pale green_; \citealp{azzam16}: "AYT2").
_Bottom panels_: Vignettes around accretion lines (see labels).
The $y$ range of the $\Pfb$($\lambda 4654$) panel applies
also to $\Pfg$ and $\Pfd$($\lambda$\la$mbda 3741, 3297$).
 (*Abb:Komboalles*)

</div>
<div id="div_fig2">

<img src="tmp_2511.10751/./Flambda_Teff_CIFIST_Bra_logg3.5.normPhot.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
Resolved continuum at $\Bra$ for $\Teff=1200$, 1400, 1600 K (_blue to red_) from the CIFIST models for $\log g=3.5$ and solar metallicity,
scaled to match, integrated over the filter, the \citet{stolker2020_I} NB4.05 photometry (see text),
and without rotational broadening (_pale lines_) or broadened to $v\sin i\approx10 $\kms$$(_darker lines_).
The _black dashed line_ is total shock emission for the fiducial case as in Figure \ref{Abb:Prof}.
 (*Abb:Atmbeitragnorm*)

</div>
<div id="div_fig3">

<img src="tmp_2511.10751/./Winkelschema_5.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Geometry for the calculation of the emission, with some important angles indicated (see text too). The point $\Vekt{P}$ on the planet is in general not in the plane of the paper, and $\alpha$ is the (true, non-projected) angle. The azimuthal angle around the planet--observer axis is denoted by $\phi$ and the azimuthal angle around the pole of the planet is denoted by $\varphi$("varphi").
Figure only partially to scale.
 (*Abb:Winkelabb*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.10751"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\nnhp}{N_2H^+ (4-3)}$
$\newcommand$
$\newcommand{\hhdp}{o-H_2D^+ (1(1,0)-1(1,1))}$
$\newcommand{\CS}{^{13}CS (8-7)}$
$\newcommand{\soo}{SO_2 19(4,16)-19(3,17)}$
$\newcommand{\textpk}[1]{\textcolor{red}{#1}}$</div>



<div id="title">

# $\large{Unveiling the Chemical Complexity and C/O Ratio of the HD 163296 Protoplanetary Disk: Constraints from Multi-line ALMA Observations of Organics, Nitriles, Sulfur-bearing, and Deuterated Molecules}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.10882-b31b1b.svg)](https://arxiv.org/abs/2511.10882)<mark>Appeared on: 2025-11-17</mark> -  _Accepted for publication in The Astrophysical Journal Supplement Series (ApJS). 43 pages, 21 figures, 4 tables_

</div>
<div id="authors">

P. Kashyap, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The physical and chemical conditions within a protoplanetary disk play a crucial role in determining its chemical composition, which is subsequently inherited by any forming planets. To probe these conditions, high-resolution molecular line observations, coupled with modelling, are essential. In this study, we investigate the chemistry of the nearby, massive, and relatively line-rich protoplanetary disk around HD 163296 using high-resolution observations from ALMA across Bands 3, 4, 6, and 7. We constrain the disk-averaged and radial distributions of column density and excitation temperature for the detected molecules using the new retrieval code DRive . The disk chemistry is modelled using the astrochemical code PEGASIS , with variations in the initial elemental C/O ratio. Our modelling, informed by molecular observations of $\ce{HCO+}$ , $\ce{DCO+}$ , $\ce{HCN}$ , $\ce{DCN}$ , $\ce{CS}$ , $\ce{HC3N}$ , $\ce{H2CO}$ , $\ce{CH3OH}$ , $\ce{HNCO}$ , and $\ce{NH2CHO}$ , allows us to place strong constraints on the C/O ratio, with a best-fit value of 1.1 that is broadly consistent with previous estimates. We present the highest-resolution DCO $^+$ emission map of this disk to date, revealing triple-ringed chemical substructures that closely align with the dust continuum rings. Additionally, our results provide the first and most stringent upper limits on the column densities of $\ce{NH2CHO}$ and $\ce{HNCO}$ in this protoplanetary disk, measured at $< 7 \times 10^{11}$ cm $^{-2}$ and $< 1 \times 10^{11}$ cm $^{-2}$ , respectively. Our chemical models suggest that $\ce{NH2CHO}$ and $\ce{HNCO}$ predominantly form on grain surfaces within the disk. However, physico-chemical desorption mechanisms are inefficient at releasing these species into detectable gas-phase abundances, yet they remain promising targets for future ALMA observations.

</div>

<div id="div_fig1">

<img src="tmp_2511.10882/./radial_profiles_tau.png" alt="Fig10" width="100%"/>

**Figure 10. -** Azimuthally averaged radial profiles (first column) and radial distributions of column density (second column), excitation temperature (third column) and the corresponding optical depth, $\tau_0$(fourth column) derived from our LTE-Slab model. In the first column, the horizontal line at the upper right of each subplot indicates the beam’s major axis. Dashed vertical lines mark continuum emission peaks at 14 au, 67 au, 101 au, and 159 au, while dotted vertical lines denote continuum depressions at 49 au, 85 au, and 145 au \citep{Huang2018a}. In the second, third, and fourth columns, grey horizontal lines show the disk-averaged column density, excitation temperature, and the $\tau_0$ for the transition that has the largest central-optical depth, respectively; their widths indicate the radial range used to generate the integrated spectrum. In the first three columns, shaded regions of the same colour represent the associated uncertainties. Note that the 10 \% calibration error and spectral-autocorrelation errors are not shown in the radial integrated-intensity profile but are included in the error bars of the column density and excitation temperature (second and third columns). Optical depths in the fourth column are calculated using equation \eqref{eq:tau_0_final} at median values of column density and excitation temperatures. (*fig:radial*)

</div>
<div id="div_fig2">

<img src="tmp_2511.10882/./cden_fit.png" alt="Fig13" width="100%"/>

**Figure 13. -** Comparison between observationally constrained, disk-averaged column densities and model-predicted radial column density distributions for varying initial carbon-to-oxygen (C/O) abundance ratios. A chi-squared analysis identifies the best-fit model with a C/O ratio of 1.1. Two chemical evolution scenarios are presented for this case: (1) an inheritance model, in which molecular abundances are inherited from a 1 Myr-old molecular cloud, and (2) a reset model, where the disk starts with primarily elemental abundances. A 20\% uncertainty is assumed for the model column densities when comparing with observations, following the methodology of \citet{Kashyap2024}. In the figure, dot-dashed vertical lines indicate the locations of continuum gaps at 49, 85, and 145 au. Horizontal dotted black lines with downward arrows show the observationally constrained upper limits of undetected molecules, while horizontal black bars (with grey shaded uncertainty regions) indicate the disk-averaged column densities of detected molecules determined from the DRive spectral line analysis (see section \ref{sec:drive-diskavg}). (*fig:cden_fit*)

</div>
<div id="div_fig3">

<img src="tmp_2511.10882/./M0.png" alt="Fig8" width="100%"/>

**Figure 8. -** Integrated intensity maps of the detected transitions. A continuum image highlighting the rings and gaps is displayed at the bottom right. The synthesised beam is indicated at the bottom left of each panel. (*fig:m0*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.10882"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

121  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
